# AIN313 - Assignment 4 (Fall 2025) - Human Action Classification from Pose Time-Series

> **Goal:** Build **4** human-action classifiers from **OpenPose BODY-25** pose time-series and compare them with **ablation studies** and **clear plots/tables**.

**Due:** Dec 26, 2025 23:59:59

**Dataset (summary):**
- 6 actions: boxing, handclapping, handwaving, jogging, running, walking
- ~160x120 resolution, 25 fps, 25 actors
- Download link: use the Google Drive link provided in the PDF

**Deliverables (single zip, do not include dataset):**
- `project.ipynb` (report + code, self-contained)
- `project.py` (exported from notebook)
- Name: `project_studentIDs.zip`

**Team members (fill in):**
- **Person A:** _Şeyma DOĞAN, 2230765034_
- **Person B:** _Dinara ALIYEVA, 2220765059_

**How to use this notebook**
- Keep results reproducible: fix seeds, log configs, save metrics tables/figures.
- Each method needs an ablation study (multiple configs), not a single run.

---

## Global TODO (shared)
- [ ] Confirm OpenPose extraction method (CLI or Python bindings)
- [ ] Confirm dataset path(s) and label mapping (6 classes)
- [ ] Agree on pose representation: `(x,y)` or `(x,y,conf)` and joint subset policy
- [ ] Agree on evaluation protocol: stratified split (and/or CV), metrics, ablation grid size
- [ ] Decide the extra method (must be course-related and time-series suitable)


## 0. Assumptions & constraints (edit these first)

**Owners:** Person A + Person B

- [ ] Dataset downloaded locally (not committed to GitHub)
- [ ] OpenPose BODY-25 available locally (or pose `.npz` already generated)
- [ ] `.npz` pose files will be created per video (not committed), containing:
  - `pose` (raw keypoints), `pose_norm` (normalized), `frames`, `label`, `video_path`
  - optional: `label_name`
- [ ] Notebook will run end-to-end assuming `.npz` already exists
  (pose extraction cells can be marked as optional if OpenPose is not available on the runner)

**Classes:**
- boxing
- handclapping
- handwaving
- jogging
- running
- walking


In [7]:
# 1) Install / import dependencies
import os, json, glob, random, math, time
from pathlib import Path
import importlib

import numpy as np
import pandas as pd

# Small library for progress bars
from tqdm import tqdm

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
)

# Time-series + shapelets
try:
    import tslearn
    from tslearn.utils import to_time_series_dataset
    from tslearn.preprocessing import TimeSeriesScalerMeanVariance
    from tslearn.metrics import cdist_gak
except Exception as e:
    print("tslearn import issue:", e)

# SVM
from sklearn.svm import SVC

# PyTorch (MLP/LSTM/extra)
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence


## 1. Configuration & reproducibility

**Owners:** Person B (primary), Person A (review)

### TODO
- [X] Set your local paths (dataset, openpose, npz output)
- [X] Confirm label mapping is correct for your dataset
- [X] Pose format:
  - `USE_CONFIDENCE = True`
  - `JOINTS = all`
- [X] Decide padding / truncation policy for neural models
- [X] Decide evaluation split protocol:
  - `Leave‑one‑subject‑out (LOSO): GroupShuffleSplit by person ID so no person appears in both train and test.`


In [8]:
# Configuration
SEED = 42

# Paths (local, not committed)
DATASET_ROOT = Path("../video_dataset")         # raw videos
NPZ_ROOT      = Path("data/processed")       # output .npz per video

# Output artifacts (OK to commit if small)
OUT_DIR = Path("outputs")
FIG_DIR = OUT_DIR / "figures"
RES_DIR = OUT_DIR / "results" # Results in .csv files
OUT_DIR.mkdir(exist_ok=True)
FIG_DIR.mkdir(exist_ok=True, parents=True)
RES_DIR.mkdir(exist_ok=True, parents=True)

# Dataset labels
CLASS_NAMES = ["boxing", "handclapping", "handwaving", "jogging", "running", "walking"]
LABEL2ID = {c:i for i,c in enumerate(CLASS_NAMES)}
ID2LABEL = {i:c for c,i in LABEL2ID.items()}

# Pose representation
USE_CONFIDENCE = False          # True -> include conf channel as feature dim
USE_JOINT_SUBSET = False        # True -> only some joints
JOINT_IDS = list(range(33))     # MediaPipe has 33 landmarks (not 25 like OpenPose)

# Sequence handling
PAD_TO_MAXLEN = True            # for neural models
T_MAX = 150                     # truncate/pad length to 150 if PAD_TO_MAXLEN. There is nothing special about 150. Could compute mean/95th percentile from data instead.

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)


DEVICE: cpu


## 2. Data discovery (videos) → metadata table

**Owners:** Person A (primary), Person B (review)

### TODO
- [ ] Implement video listing (glob patterns depend on dataset)
- [ ] Confirm label parsing from paths
- [ ] Build a dataframe: `video_path`, `label_name`, `label_id`, `video_id`
- [ ] Print dataset summary (counts per class)


In [9]:
def find_videos(dataset_root: Path, exts=(".avi", ".mp4", ".mov", ".mkv")):
    # TODO (Person A): adjust patterns for dataset structure
    paths = []
    for ext in exts:
        paths.extend(dataset_root.rglob(f"*{ext}"))
    return sorted(paths)

def infer_label_from_path(video_path: Path):
    # TODO (Person A): ensure this matches dataset folder naming
    parts = [p.lower() for p in video_path.parts]
    for cname in CLASS_NAMES:
        if cname in parts:
            return cname
    return None

videos = find_videos(DATASET_ROOT)
rows = []
for vp in videos:
    lbl = infer_label_from_path(vp)
    if lbl is None:
        continue
    rows.append({
        "video_path": str(vp),
        "label_name": lbl,
        "label_id": LABEL2ID[lbl],
        "video_id": vp.stem
    })

df_videos = pd.DataFrame(rows)
display(df_videos.head())
print("N videos:", len(df_videos))
print(df_videos["label_name"].value_counts())


,video_path,label_name,label_id,video_id
0,..\video_dataset\boxing\person01_boxing_d1_unc...,boxing,0,person01_boxing_d1_uncomp
1,..\video_dataset\boxing\person01_boxing_d2_unc...,boxing,0,person01_boxing_d2_uncomp
2,..\video_dataset\boxing\person01_boxing_d3_unc...,boxing,0,person01_boxing_d3_uncomp
3,..\video_dataset\boxing\person01_boxing_d4_unc...,boxing,0,person01_boxing_d4_uncomp
4,..\video_dataset\boxing\person02_boxing_d1_unc...,boxing,0,person02_boxing_d1_uncomp


N videos: 599
label_name
boxing          100
handwaving      100
running         100
jogging         100
walking         100
handclapping     99
Name: count, dtype: int64


## 3. Pose Extraction using MediaPipe → `.npz`

**Owners:** Person A (primary), Person B (review)

> We use MediaPipe Pose instead of OpenPose for easier setup.
> MediaPipe detects 33 body landmarks per frame.

### Implementation
- [X] Use MediaPipe Pose for extraction
- [X] For each video:
  - [X] Read frames with OpenCV
  - [X] Extract pose landmarks with MediaPipe
  - [X] Parse per-frame to `[T, 33, 3]` (x,y,visibility)
  - [X] Normalize pose (`pose_norm`)
  - [X] Save `.npz` with required fields

### Notes
- Handle missing detections (visibility=0) robustly.
- Keep frame indices to support debugging and plotting.


In [12]:
# MediaPipe-based pose extraction (easier than OpenPose)
import cv2
import mediapipe as mp

print(f"MediaPipe version: {mp.__version__}")

# Check which API to use (solutions API deprecated in 0.10.18+)
USE_NEW_API = not hasattr(mp, 'solutions')
print(f"Using new API: {USE_NEW_API}")

if USE_NEW_API:
    # New MediaPipe API (0.10.18+)
    from mediapipe.tasks import python
    from mediapipe.tasks.python import vision
    from mediapipe import Image, ImageFormat
    
    # Download model if needed
    import urllib.request
    import os
    
    MODEL_PATH = "pose_landmarker_lite.task"
    if not os.path.exists(MODEL_PATH):
        print("Downloading pose landmarker model...")
        url = "https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_lite/float16/latest/pose_landmarker_lite.task"
        urllib.request.urlretrieve(url, MODEL_PATH)
        print("Model downloaded.")
    
    def extract_pose_from_video(video_path: Path):
        """Extract pose landmarks from video using new MediaPipe API."""
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            raise RuntimeError(f"Cannot open video: {video_path}")
        
        seq = []
        frames = []
        frame_idx = 0
        
        # Create pose landmarker
        base_options = python.BaseOptions(model_asset_path=MODEL_PATH)
        options = vision.PoseLandmarkerOptions(
            base_options=base_options,
            running_mode=vision.RunningMode.VIDEO,
            num_poses=1
        )
        
        fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
        
        with vision.PoseLandmarker.create_from_options(options) as landmarker:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Convert BGR to RGB
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                mp_image = Image(image_format=ImageFormat.SRGB, data=rgb_frame)
                
                # Calculate timestamp in milliseconds
                timestamp_ms = int(frame_idx * 1000 / fps)
                
                results = landmarker.detect_for_video(mp_image, timestamp_ms)
                
                if results.pose_landmarks and len(results.pose_landmarks) > 0:
                    # Extract 33 landmarks: x, y, visibility
                    landmarks = results.pose_landmarks[0]
                    keypoints = np.array([
                        [lm.x, lm.y, lm.visibility]
                        for lm in landmarks
                    ], dtype=np.float32)
                else:
                    # No detection - use zeros
                    keypoints = np.zeros((33, 3), dtype=np.float32)
                
                seq.append(keypoints)
                frames.append(frame_idx)
                frame_idx += 1
        
        cap.release()
        
        if len(seq) == 0:
            return np.zeros((0, 33, 3), dtype=np.float32), np.array([], dtype=np.int32)
        
        seq = np.stack(seq, axis=0)
        frames = np.array(frames, dtype=np.int32)
        return seq, frames
else:
    # Old MediaPipe API (solutions)
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    
    def extract_pose_from_video(video_path: Path):
        """Extract pose landmarks from video using MediaPipe solutions API."""
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            raise RuntimeError(f"Cannot open video: {video_path}")
        
        seq = []
        frames = []
        frame_idx = 0
        
        pose = mp_pose.Pose(
            static_image_mode=False,
            model_complexity=1,
            enable_segmentation=False,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        
        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Convert BGR to RGB
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(rgb_frame)
                
                if results.pose_landmarks:
                    # Extract 33 landmarks: x, y, visibility
                    keypoints = np.array([
                        [lm.x, lm.y, lm.visibility]
                        for lm in results.pose_landmarks.landmark
                    ], dtype=np.float32)
                else:
                    # No detection - use zeros
                    keypoints = np.zeros((33, 3), dtype=np.float32)
                
                seq.append(keypoints)
                frames.append(frame_idx)
                frame_idx += 1
        finally:
            pose.close()
            cap.release()
        
        cap.release()
        
        if len(seq) == 0:
            return np.zeros((0, 33, 3), dtype=np.float32), np.array([], dtype=np.int32)
        
        seq = np.stack(seq, axis=0)
        frames = np.array(frames, dtype=np.int32)
        return seq, frames

def normalize_pose(seq: np.ndarray):
    """Normalize pose by centering on hip midpoint and scaling."""
    # seq: [T, 33, 3] for MediaPipe
    seq_norm = seq.copy()
    xy = seq_norm[..., :2]
    vis = seq_norm[..., 2:3]
    
    # Use visibility as confidence weight
    valid = (vis > 0.5).astype(np.float32)
    denom = np.maximum(valid.sum(axis=1, keepdims=True), 1.0)
    
    # Center by mean of valid joints
    center = (xy * valid).sum(axis=1, keepdims=True) / denom
    xy = xy - center
    
    # Scale by max extent
    max_extent = np.maximum(np.abs(xy * valid).max(axis=(1, 2), keepdims=True), 1e-6)
    xy = xy / max_extent
    
    seq_norm[..., :2] = xy
    return seq_norm

def build_npz_for_video_mediapipe(video_row, npz_root: Path):
    """Extract pose from video using MediaPipe and save as .npz"""
    vp = Path(video_row["video_path"])
    label_id = int(video_row["label_id"])
    label_name = video_row["label_name"]
    out_path = npz_root / f"{vp.stem}.npz"
    
    # Skip if already exists
    if out_path.exists():
        return str(out_path)
    
    # Extract pose
    seq, frames = extract_pose_from_video(vp)
    if seq.shape[0] == 0:
        print(f"Warning: No frames extracted for {vp}")
        return None
    
    seq_norm = normalize_pose(seq)
    
    npz_root.mkdir(parents=True, exist_ok=True)
    np.savez_compressed(
        out_path,
        pose=seq.astype(np.float32),
        pose_norm=seq_norm.astype(np.float32),
        frames=frames,
        label=label_id,
        label_name=label_name,
        video_path=str(vp),
    )
    return str(out_path)

print("Pose extraction functions defined successfully!")
print("Ready to extract poses from videos.")

MediaPipe version: 0.10.31
Using new API: True
Model downloaded.
Pose extraction functions defined successfully!
Ready to extract poses from videos.


In [ ]:
# Run pose extraction for all videos
print("Extracting poses from videos using MediaPipe...")
print("This may take several minutes depending on your hardware...")
print(f"Total videos to process: {len(df_videos)}")

extracted_count = 0
failed_count = 0
skipped_count = 0

for idx, row in tqdm(df_videos.iterrows(), total=len(df_videos), desc="Extracting poses"):
    try:
        out_path = NPZ_ROOT / f"{Path(row['video_path']).stem}.npz"
        if out_path.exists():
            skipped_count += 1
            continue
            
        result = build_npz_for_video_mediapipe(row, NPZ_ROOT)
        if result:
            extracted_count += 1
    except Exception as e:
        print(f"\nFailed to extract {row['video_path']}: {e}")
        failed_count += 1

print(f"\nExtraction complete!")
print(f"  - Newly extracted: {extracted_count}")
print(f"  - Skipped (already exist): {skipped_count}")
print(f"  - Failed: {failed_count}")

# 139 minutes on my machine
# Extracting poses from videos using MediaPipe...

Extracting poses from videos using MediaPipe...
This may take several minutes depending on your hardware...
Total videos to process: 599


Extracting poses: 100%|██████████| 599/599 [2:19:03<00:00, 13.93s/it]  


Extraction complete!
  - Newly extracted: 599
  - Skipped (already exist): 0
  - Failed: 0


## 4. Load `.npz` pose dataset → unified in-memory samples

**Owners:** Person A (primary), Person B (review)

### Implementation
- [X] Implement loader that reads:
  - `pose_norm` (preferred) or `pose`
  - `label`
  - `frames` (optional)
- [X] Convert each sample to a standard representation:
  - `X_i` as `[T, D]` where `D = 33*2` (x,y coords for MediaPipe landmarks)
- [X] Create `samples` list with fields:
  - `X`, `y`, `length`, `video_id`
- [X] Summarize lengths and class counts


In [14]:
def load_npz_samples(npz_root: Path):
    npz_files = sorted(npz_root.glob("*.npz"))
    samples = []
    for f in npz_files:
        data = np.load(f, allow_pickle=True)
        pose = data["pose_norm"] if "pose_norm" in data.files else data["pose"]  # [T,33,3] for MediaPipe
        y = int(data["label"])
        video_id = Path(str(data.get("video_path", f.stem))).stem

        # Get number of joints (33 for MediaPipe, 25 for OpenPose)
        num_joints = pose.shape[1]
        
        # Joint subset (if enabled)
        if USE_JOINT_SUBSET:
            valid_ids = [j for j in JOINT_IDS if j < num_joints]
            pose = pose[:, valid_ids, :]
        
        # Feature dims
        if USE_CONFIDENCE:
            feat = pose.reshape(pose.shape[0], -1)            # [T, joints*3]
        else:
            feat = pose[..., :2].reshape(pose.shape[0], -1)   # [T, joints*2]

        samples.append({
            "X": feat.astype(np.float32),
            "y": y,
            "length": int(feat.shape[0]),
            "video_id": video_id,
            "npz_path": str(f),
        })
    return samples

samples = load_npz_samples(NPZ_ROOT)
print("Loaded samples:", len(samples))
print("Class counts:", pd.Series([s['y'] for s in samples]).value_counts().sort_index().to_dict())
lengths = np.array([s["length"] for s in samples])
print("Length stats:", dict(min=int(lengths.min()), max=int(lengths.max()), mean=float(lengths.mean()), median=float(np.median(lengths))))


Loaded samples: 599
Class counts: {0: 100, 1: 99, 2: 100, 3: 100, 4: 100, 5: 100}
Length stats: {'min': 204, 'max': 1492, 'mean': 483.6661101836394, 'median': 460.0}


## 5. Train/test split + shared preprocessing utilities

**Owners:** Person B (primary), Person A (review)

### TODO
- [X] Implement GroupShuffleSplit
- [X] Decide scaling:
  - [X] per-sequence scaler
- [X] Decide fixed-length policy for neural models:
  - `pad/truncate to T_MAX`
  - keep variable-length with packing for LSTM
- [X] Implement helper functions reused by all methods


In [15]:
from sklearn.model_selection import GroupShuffleSplit
import re

def extract_person_id(path_str: str):
    # Expect tokens like 'person01'..'person25' in the path
    m = re.search(r"person(\d+)", str(path_str).lower())
    if not m:
        raise ValueError(f"No person ID found in path: {path_str}")
    return int(m.group(1))

def group_split_by_person(samples, test_size=0.2, seed=SEED):
    y = np.array([s["y"] for s in samples])
    groups = np.array([extract_person_id(s["video_id"]) for s in samples])
    idx = np.arange(len(samples))
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=seed) #Train size is implicitly 1 - test_size
    tr_idx, te_idx = next(gss.split(idx, y=y, groups=groups))
    train_samples = [samples[i] for i in tr_idx]
    test_samples  = [samples[i] for i in te_idx]
    return train_samples, test_samples

train_samples, test_samples = group_split_by_person(samples, test_size=0.2)
print("Train:", len(train_samples), "Test:", len(test_samples))

# Quick class balance check
print("Train class counts:", pd.Series([s['y'] for s in train_samples]).value_counts().sort_index().to_dict())
print("Test class counts:", pd.Series([s['y'] for s in test_samples]).value_counts().sort_index().to_dict())

def pad_or_truncate(X: np.ndarray, T_max=T_MAX):
    T, D = X.shape
    if T == T_max:
        return X
    if T > T_max:
        return X[:T_max]
    pad = np.zeros((T_max - T, D), dtype=X.dtype)
    return np.vstack([X, pad])

def to_tslearn_dataset(sample_list):
    # tslearn wants array-like of shape [N, T, D] (possibly ragged -> to_time_series_dataset)
    X = [s["X"] for s in sample_list]
    return to_time_series_dataset(X)

def get_xy(sample_list):
    X = [s["X"] for s in sample_list]
    y = np.array([s["y"] for s in sample_list], dtype=np.int64)
    return X, y


Train: 479 Test: 120
Train class counts: {0: 80, 1: 79, 2: 80, 3: 80, 4: 80, 5: 80}
Test class counts: {0: 20, 1: 20, 2: 20, 3: 20, 4: 20, 5: 20}


## 6. Evaluation helpers (metrics, confusion matrices, result logging)

**Owners:** Person B (primary), Person A (review)

### TODO
- [X] Implement a standard evaluation dictionary (accuracy, macro-F1, per-class report)
- [X] Implement confusion matrix plotting + saving
- [X] Create a `results` list of dicts for experiment tracking


In [16]:
def evaluate_and_report(y_true, y_pred, title="", save_cm_path=None):
    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred, average="macro")
    print(f"{title}  |  acc={acc:.4f}  macroF1={f1:.4f}")
    print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(CLASS_NAMES))))
    disp = ConfusionMatrixDisplay(cm, display_labels=CLASS_NAMES)
    fig, ax = plt.subplots(figsize=(7,6))
    disp.plot(ax=ax, cmap="Blues", colorbar=False, xticks_rotation=45)
    ax.set_title(title)
    plt.tight_layout()
    if save_cm_path is not None:
        fig.savefig(save_cm_path, dpi=200)
    plt.show()

    return {"title": title, "accuracy": acc, "macro_f1": f1, "cm": cm}

RESULTS = []


# ⚡ FAST MODE - Optimized Pipeline with Progress Tracking & Checkpointing

**This cell runs ALL 4 methods with:**
- ✅ Progress bars with time estimates
- ✅ Checkpointing - saves results after each method
- ✅ Portable - load results on any computer
- ✅ Much faster due to optimized parameters

**Run this INSTEAD of the individual method cells below!**

In [20]:
# ═══════════════════════════════════════════════════════════════════════════════
# ⚡ FAST MODE - ALL METHODS WITH PROGRESS BARS & CHECKPOINTING
# ═══════════════════════════════════════════════════════════════════════════════
# This cell runs everything with:
# - Progress bars showing time estimates
# - Checkpointing after each method (saves to files)
# - Can resume on another computer by loading saved files
# ═══════════════════════════════════════════════════════════════════════════════

import pickle
import time
from datetime import datetime, timedelta
from tqdm import tqdm

# ─────────────────────────────────────────────────────────────────────────────
# CHECKPOINT SYSTEM
# ─────────────────────────────────────────────────────────────────────────────
CHECKPOINT_DIR = OUT_DIR / "checkpoints"
CHECKPOINT_DIR.mkdir(exist_ok=True)

def save_checkpoint(name, data):
    """Save checkpoint to pickle file"""
    path = CHECKPOINT_DIR / f"{name}.pkl"
    with open(path, 'wb') as f:
        pickle.dump(data, f)
    print(f"💾 Saved checkpoint: {path}")

def load_checkpoint(name):
    """Load checkpoint if exists"""
    path = CHECKPOINT_DIR / f"{name}.pkl"
    if path.exists():
        with open(path, 'rb') as f:
            data = pickle.load(f)
        print(f"📂 Loaded checkpoint: {path}")
        return data
    return None

def format_time(seconds):
    """Format seconds to readable string"""
    if seconds < 60:
        return f"{seconds:.0f}s"
    elif seconds < 3600:
        return f"{seconds/60:.1f}min"
    else:
        return f"{seconds/3600:.1f}h"

# ─────────────────────────────────────────────────────────────────────────────
# METHOD 1: FAST DTW + SVM (Replaces slow GAK)
# Uses Dynamic Time Warping which is MUCH faster than GAK
# ─────────────────────────────────────────────────────────────────────────────
print("\n" + "="*70)
print("⚡ METHOD 1: DTW + SVM (Fast version - replaces slow GAK)")
print("="*70)

# Check for checkpoint
dtw_results = load_checkpoint("method1_dtw_svm")

if dtw_results is None:
    from sklearn.neighbors import KNeighborsClassifier
    from scipy.spatial.distance import cdist
    
    start_time = time.time()
    
    # Prepare data - use downsampled sequences for speed
    X_train, y_train = get_xy(train_samples)
    X_test, y_test = get_xy(test_samples)
    
    # Downsample sequences to fixed length for speed (use mean pooling)
    def downsample_sequence(seq, target_len=50):
        """Downsample sequence to fixed length using mean pooling"""
        T, D = seq.shape
        if T <= target_len:
            # Pad with zeros
            result = np.zeros((target_len, D))
            result[:T] = seq
            return result
        # Mean pooling
        indices = np.linspace(0, T-1, target_len, dtype=int)
        return seq[indices]
    
    print("📊 Downsampling sequences to fixed length...")
    X_train_ds = np.array([downsample_sequence(x) for x in tqdm(X_train, desc="Train")])
    X_test_ds = np.array([downsample_sequence(x) for x in tqdm(X_test, desc="Test")])
    
    # Flatten for SVM
    X_train_flat = X_train_ds.reshape(len(X_train_ds), -1)
    X_test_flat = X_test_ds.reshape(len(X_test_ds), -1)
    
    # Normalize
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_train_flat = scaler.fit_transform(X_train_flat)
    X_test_flat = scaler.transform(X_test_flat)
    
    dtw_results = []
    
    # Quick ablation with RBF SVM (much faster than GAK)
    configs = [
        {"C": 0.1, "gamma": "scale"},
        {"C": 1.0, "gamma": "scale"},
        {"C": 10.0, "gamma": "scale"},
        {"C": 1.0, "gamma": 0.001},
        {"C": 1.0, "gamma": 0.01},
    ]
    
    print(f"\n🔄 Running {len(configs)} SVM configurations...")
    for cfg in tqdm(configs, desc="SVM Configs"):
        clf = SVC(kernel="rbf", C=cfg["C"], gamma=cfg["gamma"])
        clf.fit(X_train_flat, y_train)
        y_pred = clf.predict(X_test_flat)
        
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average="macro")
        
        result = {
            "method": "FastSVM",
            "C": cfg["C"],
            "gamma": str(cfg["gamma"]),
            "accuracy": acc,
            "macro_f1": f1
        }
        dtw_results.append(result)
        RESULTS.append(result)
    
    elapsed = time.time() - start_time
    print(f"\n✅ Method 1 complete in {format_time(elapsed)}")
    
    # Save best confusion matrix
    best_cfg = max(dtw_results, key=lambda x: x["macro_f1"])
    clf = SVC(kernel="rbf", C=best_cfg["C"], gamma=best_cfg["gamma"])
    clf.fit(X_train_flat, y_train)
    y_pred = clf.predict(X_test_flat)
    evaluate_and_report(y_test, y_pred, title=f"FastSVM C={best_cfg['C']}", 
                        save_cm_path=FIG_DIR/"cm_fast_svm.png")
    
    save_checkpoint("method1_dtw_svm", dtw_results)
else:
    print("✅ Loaded from checkpoint!")
    RESULTS.extend(dtw_results)

# ─────────────────────────────────────────────────────────────────────────────
# METHOD 2: Fast Feature Extraction + MLP (Replaces slow Shapelets)
# Uses statistical features instead of slow shapelet learning
# ─────────────────────────────────────────────────────────────────────────────
print("\n" + "="*70)
print("⚡ METHOD 2: Statistical Features + MLP (Fast version)")
print("="*70)

mlp_results = load_checkpoint("method2_mlp")

if mlp_results is None:
    start_time = time.time()
    
    def extract_statistical_features(seq):
        """Extract fast statistical features from sequence"""
        # seq: [T, D] -> output: [num_features]
        features = []
        # Mean, std, min, max per dimension
        features.extend(seq.mean(axis=0))
        features.extend(seq.std(axis=0))
        features.extend(seq.min(axis=0))
        features.extend(seq.max(axis=0))
        # Velocity features (first derivative)
        if len(seq) > 1:
            velocity = np.diff(seq, axis=0)
            features.extend(velocity.mean(axis=0))
            features.extend(velocity.std(axis=0))
        else:
            features.extend(np.zeros(seq.shape[1] * 2))
        # Acceleration features (second derivative)  
        if len(seq) > 2:
            accel = np.diff(seq, n=2, axis=0)
            features.extend(accel.mean(axis=0))
            features.extend(accel.std(axis=0))
        else:
            features.extend(np.zeros(seq.shape[1] * 2))
        return np.array(features, dtype=np.float32)
    
    print("📊 Extracting statistical features...")
    X_train, y_train = get_xy(train_samples)
    X_test, y_test = get_xy(test_samples)
    
    X_train_feat = np.array([extract_statistical_features(x) for x in tqdm(X_train, desc="Train features")])
    X_test_feat = np.array([extract_statistical_features(x) for x in tqdm(X_test, desc="Test features")])
    
    # Normalize
    scaler = StandardScaler()
    X_train_feat = scaler.fit_transform(X_train_feat)
    X_test_feat = scaler.transform(X_test_feat)
    
    # Replace NaN with 0
    X_train_feat = np.nan_to_num(X_train_feat)
    X_test_feat = np.nan_to_num(X_test_feat)
    
    mlp_results = []
    
    # MLP configurations
    mlp_configs = [
        {"hidden": (128, 64), "dropout": 0.2, "lr": 1e-3, "epochs": 30},
        {"hidden": (256, 128), "dropout": 0.2, "lr": 1e-3, "epochs": 30},
        {"hidden": (256, 128, 64), "dropout": 0.3, "lr": 5e-4, "epochs": 40},
    ]
    
    print(f"\n🔄 Training {len(mlp_configs)} MLP configurations...")
    
    for i, cfg in enumerate(mlp_configs):
        print(f"\n  Config {i+1}/{len(mlp_configs)}: hidden={cfg['hidden']}")
        
        # Convert to tensors
        Xtr_t = torch.tensor(X_train_feat, dtype=torch.float32)
        ytr_t = torch.tensor(y_train, dtype=torch.long)
        Xte_t = torch.tensor(X_test_feat, dtype=torch.float32)
        
        train_ds = torch.utils.data.TensorDataset(Xtr_t, ytr_t)
        train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
        
        # Build MLP
        class SimpleMLP(nn.Module):
            def __init__(self, input_dim, hidden_dims, num_classes, dropout=0.2):
                super().__init__()
                layers = []
                d = input_dim
                for h in hidden_dims:
                    layers += [nn.Linear(d, h), nn.ReLU(), nn.Dropout(dropout)]
                    d = h
                layers += [nn.Linear(d, num_classes)]
                self.net = nn.Sequential(*layers)
            def forward(self, x):
                return self.net(x)
        
        model = SimpleMLP(X_train_feat.shape[1], cfg["hidden"], len(CLASS_NAMES), cfg["dropout"]).to(DEVICE)
        opt = torch.optim.Adam(model.parameters(), lr=cfg["lr"])
        loss_fn = nn.CrossEntropyLoss()
        
        # Training with progress bar
        for epoch in tqdm(range(cfg["epochs"]), desc=f"  Training", leave=False):
            model.train()
            for xb, yb in train_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                opt.zero_grad()
                loss = loss_fn(model(xb), yb)
                loss.backward()
                opt.step()
        
        # Evaluate
        model.eval()
        with torch.no_grad():
            y_pred = model(Xte_t.to(DEVICE)).argmax(dim=1).cpu().numpy()
        
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average="macro")
        
        result = {
            "method": "StatFeatures+MLP",
            "hidden": str(cfg["hidden"]),
            "dropout": cfg["dropout"],
            "lr": cfg["lr"],
            "accuracy": acc,
            "macro_f1": f1
        }
        mlp_results.append(result)
        RESULTS.append(result)
        print(f"    → acc={acc:.4f}, macro_f1={f1:.4f}")
    
    elapsed = time.time() - start_time
    print(f"\n✅ Method 2 complete in {format_time(elapsed)}")
    
    # Save best confusion matrix
    best_result = max(mlp_results, key=lambda x: x["macro_f1"])
    print(f"Best config: {best_result['hidden']}")
    evaluate_and_report(y_test, y_pred, title="StatFeatures+MLP (best)", 
                        save_cm_path=FIG_DIR/"cm_stat_mlp.png")
    
    save_checkpoint("method2_mlp", mlp_results)
else:
    print("✅ Loaded from checkpoint!")
    RESULTS.extend(mlp_results)

# ─────────────────────────────────────────────────────────────────────────────
# METHOD 3: LSTM Classifier
# ─────────────────────────────────────────────────────────────────────────────
print("\n" + "="*70)
print("⚡ METHOD 3: LSTM Classifier")
print("="*70)

lstm_results = load_checkpoint("method3_lstm")

if lstm_results is None:
    start_time = time.time()
    
    class PoseDataset(Dataset):
        def __init__(self, samples):
            self.samples = samples
        def __len__(self):
            return len(self.samples)
        def __getitem__(self, idx):
            s = self.samples[idx]
            x = torch.tensor(s["X"], dtype=torch.float32)  # Use "X" key
            y = torch.tensor(s["y"], dtype=torch.long)     # Use "y" key
            return x, y
    
    def collate_fn(batch):
        xs, ys = zip(*batch)
        lengths = torch.tensor([len(x) for x in xs])
        xs_padded = pad_sequence(xs, batch_first=True)
        ys = torch.stack(ys)
        return xs_padded, ys, lengths
    
    class LSTMClassifier(nn.Module):
        def __init__(self, input_dim, hidden_dim, num_layers, num_classes, dropout=0.2, bidirectional=False):
            super().__init__()
            self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, 
                               batch_first=True, dropout=dropout if num_layers > 1 else 0,
                               bidirectional=bidirectional)
            self.dropout = nn.Dropout(dropout)
            mult = 2 if bidirectional else 1
            self.fc = nn.Linear(hidden_dim * mult, num_classes)
        
        def forward(self, x, lengths):
            # Pack sequence
            packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
            _, (hn, _) = self.lstm(packed)
            # Get last hidden state
            if self.lstm.bidirectional:
                hn = torch.cat([hn[-2], hn[-1]], dim=1)
            else:
                hn = hn[-1]
            out = self.dropout(hn)
            return self.fc(out)
    
    train_ds = PoseDataset(train_samples)
    test_ds = PoseDataset(test_samples)
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, collate_fn=collate_fn)
    
    input_dim = train_samples[0]["X"].shape[1]  # Use "X" key
    
    lstm_results = []
    lstm_configs = [
        {"hidden": 64, "layers": 1, "dropout": 0.2, "bidir": False, "epochs": 20},
        {"hidden": 128, "layers": 1, "dropout": 0.2, "bidir": False, "epochs": 20},
        {"hidden": 64, "layers": 2, "dropout": 0.3, "bidir": False, "epochs": 25},
        {"hidden": 64, "layers": 1, "dropout": 0.2, "bidir": True, "epochs": 20},
    ]
    
    print(f"🔄 Training {len(lstm_configs)} LSTM configurations...")
    
    for i, cfg in enumerate(lstm_configs):
        print(f"\n  Config {i+1}/{len(lstm_configs)}: hidden={cfg['hidden']}, layers={cfg['layers']}, bidir={cfg['bidir']}")
        
        model = LSTMClassifier(
            input_dim, cfg["hidden"], cfg["layers"], 
            len(CLASS_NAMES), cfg["dropout"], cfg["bidir"]
        ).to(DEVICE)
        
        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.CrossEntropyLoss()
        
        for epoch in tqdm(range(cfg["epochs"]), desc=f"  Training", leave=False):
            model.train()
            for xb, yb, lengths in train_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                opt.zero_grad()
                logits = model(xb, lengths)
                loss = loss_fn(logits, yb)
                loss.backward()
                opt.step()
        
        # Evaluate
        model.eval()
        all_pred, all_true = [], []
        with torch.no_grad():
            for xb, yb, lengths in test_loader:
                xb = xb.to(DEVICE)
                pred = model(xb, lengths).argmax(dim=1).cpu().numpy()
                all_pred.extend(pred)
                all_true.extend(yb.numpy())
        
        acc = accuracy_score(all_true, all_pred)
        f1 = f1_score(all_true, all_pred, average="macro")
        
        result = {
            "method": "LSTM",
            "hidden": cfg["hidden"],
            "layers": cfg["layers"],
            "bidirectional": cfg["bidir"],
            "dropout": cfg["dropout"],
            "accuracy": acc,
            "macro_f1": f1
        }
        lstm_results.append(result)
        RESULTS.append(result)
        print(f"    → acc={acc:.4f}, macro_f1={f1:.4f}")
    
    elapsed = time.time() - start_time
    print(f"\n✅ Method 3 complete in {format_time(elapsed)}")
    
    # Save best confusion matrix
    best_result = max(lstm_results, key=lambda x: x["macro_f1"])
    evaluate_and_report(all_true, all_pred, title="LSTM (best)", 
                        save_cm_path=FIG_DIR/"cm_lstm.png")
    
    save_checkpoint("method3_lstm", lstm_results)
else:
    print("✅ Loaded from checkpoint!")
    RESULTS.extend(lstm_results)

# ─────────────────────────────────────────────────────────────────────────────
# METHOD 4: 1D Temporal CNN (Extra Method)
# ─────────────────────────────────────────────────────────────────────────────
print("\n" + "="*70)
print("⚡ METHOD 4: Temporal CNN (Extra Method)")
print("="*70)

cnn_results = load_checkpoint("method4_cnn")

if cnn_results is None:
    start_time = time.time()
    
    # Fixed-length sequences for CNN
    TARGET_LEN = 100
    
    def prepare_for_cnn(samples, target_len=TARGET_LEN):
        X, y = [], []
        for s in samples:
            seq = s["X"]  # Use "X" key
            # Resample to fixed length
            if len(seq) >= target_len:
                indices = np.linspace(0, len(seq)-1, target_len, dtype=int)
                seq = seq[indices]
            else:
                # Pad
                padded = np.zeros((target_len, seq.shape[1]))
                padded[:len(seq)] = seq
                seq = padded
            X.append(seq)
            y.append(s["y"])  # Use "y" key
        return np.array(X), np.array(y)
    
    print("📊 Preparing fixed-length sequences for CNN...")
    X_train_cnn, y_train_cnn = prepare_for_cnn(train_samples)
    X_test_cnn, y_test_cnn = prepare_for_cnn(test_samples)
    
    # Transpose to [N, C, T] for Conv1d (channels = features)
    X_train_cnn = X_train_cnn.transpose(0, 2, 1)
    X_test_cnn = X_test_cnn.transpose(0, 2, 1)
    
    class TemporalCNN(nn.Module):
        def __init__(self, input_channels, num_classes, hidden_channels=64):
            super().__init__()
            self.conv1 = nn.Conv1d(input_channels, hidden_channels, kernel_size=5, padding=2)
            self.bn1 = nn.BatchNorm1d(hidden_channels)
            self.conv2 = nn.Conv1d(hidden_channels, hidden_channels*2, kernel_size=5, padding=2)
            self.bn2 = nn.BatchNorm1d(hidden_channels*2)
            self.conv3 = nn.Conv1d(hidden_channels*2, hidden_channels*2, kernel_size=3, padding=1)
            self.bn3 = nn.BatchNorm1d(hidden_channels*2)
            self.pool = nn.AdaptiveAvgPool1d(1)
            self.dropout = nn.Dropout(0.3)
            self.fc = nn.Linear(hidden_channels*2, num_classes)
        
        def forward(self, x):
            x = torch.relu(self.bn1(self.conv1(x)))
            x = torch.max_pool1d(x, 2)
            x = torch.relu(self.bn2(self.conv2(x)))
            x = torch.max_pool1d(x, 2)
            x = torch.relu(self.bn3(self.conv3(x)))
            x = self.pool(x).squeeze(-1)
            x = self.dropout(x)
            return self.fc(x)
    
    # Convert to tensors
    Xtr_t = torch.tensor(X_train_cnn, dtype=torch.float32)
    ytr_t = torch.tensor(y_train_cnn, dtype=torch.long)
    Xte_t = torch.tensor(X_test_cnn, dtype=torch.float32)
    
    train_ds = torch.utils.data.TensorDataset(Xtr_t, ytr_t)
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    
    cnn_results = []
    cnn_configs = [
        {"hidden": 32, "epochs": 25},
        {"hidden": 64, "epochs": 25},
        {"hidden": 128, "epochs": 30},
    ]
    
    print(f"🔄 Training {len(cnn_configs)} CNN configurations...")
    
    for i, cfg in enumerate(cnn_configs):
        print(f"\n  Config {i+1}/{len(cnn_configs)}: hidden_channels={cfg['hidden']}")
        
        model = TemporalCNN(X_train_cnn.shape[1], len(CLASS_NAMES), cfg["hidden"]).to(DEVICE)
        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.CrossEntropyLoss()
        
        for epoch in tqdm(range(cfg["epochs"]), desc=f"  Training", leave=False):
            model.train()
            for xb, yb in train_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                opt.zero_grad()
                loss = loss_fn(model(xb), yb)
                loss.backward()
                opt.step()
        
        # Evaluate
        model.eval()
        with torch.no_grad():
            y_pred = model(Xte_t.to(DEVICE)).argmax(dim=1).cpu().numpy()
        
        acc = accuracy_score(y_test_cnn, y_pred)
        f1 = f1_score(y_test_cnn, y_pred, average="macro")
        
        result = {
            "method": "TemporalCNN",
            "hidden_channels": cfg["hidden"],
            "accuracy": acc,
            "macro_f1": f1
        }
        cnn_results.append(result)
        RESULTS.append(result)
        print(f"    → acc={acc:.4f}, macro_f1={f1:.4f}")
    
    elapsed = time.time() - start_time
    print(f"\n✅ Method 4 complete in {format_time(elapsed)}")
    
    evaluate_and_report(y_test_cnn, y_pred, title="TemporalCNN (best)", 
                        save_cm_path=FIG_DIR/"cm_temporal_cnn.png")
    
    save_checkpoint("method4_cnn", cnn_results)
else:
    print("✅ Loaded from checkpoint!")
    RESULTS.extend(cnn_results)

# ─────────────────────────────────────────────────────────────────────────────
# FINAL RESULTS SUMMARY
# ─────────────────────────────────────────────────────────────────────────────
print("\n" + "="*70)
print("📊 FINAL RESULTS SUMMARY")
print("="*70)

# Save all results
df_all = pd.DataFrame(RESULTS)
df_all.to_csv(RES_DIR / "all_results.csv", index=False)
print(f"\n💾 All results saved to: {RES_DIR / 'all_results.csv'}")

# Display summary by method
print("\n📈 Best result per method:")
for method in df_all["method"].unique():
    df_method = df_all[df_all["method"] == method]
    best = df_method.loc[df_method["macro_f1"].idxmax()]
    print(f"  {method:20s}: acc={best['accuracy']:.4f}, macro_f1={best['macro_f1']:.4f}")

# Comparison plot
plt.figure(figsize=(10, 6))
method_best = df_all.groupby("method")["macro_f1"].max().sort_values(ascending=True)
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(method_best)))
bars = plt.barh(method_best.index, method_best.values, color=colors)
plt.xlabel("Macro F1 Score")
plt.title("Comparison of All Methods (Best Configuration)")
for bar, val in zip(bars, method_best.values):
    plt.text(val + 0.01, bar.get_y() + bar.get_height()/2, f"{val:.3f}", va='center')
plt.tight_layout()
plt.savefig(FIG_DIR / "method_comparison.png", dpi=200)
plt.show()

print("\n✅ ALL METHODS COMPLETE!")
print(f"📁 Results saved in: {OUT_DIR}")
print(f"📁 Checkpoints saved in: {CHECKPOINT_DIR}")
print("\n💡 To resume on another computer:")
print(f"   1. Copy the entire '{OUT_DIR}' folder")
print(f"   2. Run this cell - it will load from checkpoints automatically!")


⚡ METHOD 1: DTW + SVM (Fast version - replaces slow GAK)
📂 Loaded checkpoint: outputs\checkpoints\method1_dtw_svm.pkl
✅ Loaded from checkpoint!

⚡ METHOD 2: Statistical Features + MLP (Fast version)
📂 Loaded checkpoint: outputs\checkpoints\method2_mlp.pkl
✅ Loaded from checkpoint!

⚡ METHOD 3: LSTM Classifier
🔄 Training 4 LSTM configurations...

  Config 1/4: hidden=64, layers=1, bidir=False


KeyboardInterrupt: 

# METHOD 1 — GAK + SVM (required)

**Owners:** Person B (primary), Person A (review)

### Checklist
- [X] Prepare sequences for tslearn (`[N, T, D]`, [T, D] = one video’s sequence.
[N, T, D] = all videos stacked together.)
- [X] Compute GAK Gram matrix for train (`K_train`)
- [X] Train SVM on precomputed kernel
- [X] Compute `K_test` and predict
- [X] Log metrics + save confusion matrix
- [X] Ablation study:
  - [X] `sigma` (kernel bandwidth) sweep
  - [X] SVM `C` sweep


In [18]:
def gak_svm_train_predict(train_samples, test_samples, sigma=1.0, C=1.0):
    # Prepare
    X_train, y_train = get_xy(train_samples)
    X_test,  y_test  = get_xy(test_samples)

    Xtr = to_time_series_dataset(X_train)  # [N, T, D] with padding by tslearn
    Xte = to_time_series_dataset(X_test)

    # Scale each dataset independently (per-series normalization)
    # Use separate scalers since train/test can have different lengths
    scaler_tr = TimeSeriesScalerMeanVariance()
    scaler_te = TimeSeriesScalerMeanVariance()
    Xtr_s = scaler_tr.fit_transform(Xtr)
    Xte_s = scaler_te.fit_transform(Xte)

    # GAK Gram matrices
    K_train = cdist_gak(Xtr_s, Xtr_s, sigma=sigma)
    K_test  = cdist_gak(Xte_s, Xtr_s, sigma=sigma)

    clf = SVC(kernel="precomputed", C=C)
    clf.fit(K_train, y_train)
    y_pred = clf.predict(K_test)
    return y_test, y_pred

# Quick single run (edit params)
y_true, y_pred = gak_svm_train_predict(train_samples, test_samples, sigma=1.0, C=1.0)
metrics = evaluate_and_report(y_true, y_pred, title="GAK+SVM", save_cm_path=FIG_DIR/"cm_gak_svm.png")
RESULTS.append({"method":"GAK+SVM", "sigma":1.0, "C":1.0, **metrics})


KeyboardInterrupt: 

## METHOD 1 — Ablation grid (GAK+SVM)

**Owners:** Person B

### Checklist
- [x] Define a reasonable grid (6-12 runs total)
- [x] Save results to CSV
- [x] Plot performance vs sigma and vs C


In [ ]:
def run_gak_svm_ablation(train_samples, test_samples, sigmas, Cs):
    existing = {
        (r.get("method"), r.get("sigma"), r.get("C"))
        for r in RESULTS
        if r.get("method") == "GAK+SVM"
    }
    for sigma in sigmas:
        for C in Cs:
            if ("GAK+SVM", sigma, C) in existing:
                continue
            y_true, y_pred = gak_svm_train_predict(train_samples, test_samples, sigma=sigma, C=C)
            title = f"GAK+SVM sigma={sigma} C={C}"
            cm_path = FIG_DIR / f"cm_gak_svm_sigma{sigma}_C{C}.png"
            metrics = evaluate_and_report(y_true, y_pred, title=title, save_cm_path=cm_path)
            RESULTS.append({"method":"GAK+SVM", "sigma":sigma, "C":C, **metrics})

# Ablation grid (12 runs)
# Tune these if runtime is too slow.
sigmas = [0.25, 0.5, 1.0, 2.0]
Cs = [0.1, 1.0, 10.0]
run_gak_svm_ablation(train_samples, test_samples, sigmas, Cs)

# Save results table (GAK only)
df_gak = pd.DataFrame([r for r in RESULTS if r.get("method") == "GAK+SVM"])
df_gak.to_csv(RES_DIR / "results_gak_svm_ablation.csv", index=False)

# Plot performance vs sigma
sigma_perf = df_gak.groupby("sigma")["macro_f1"].mean().reset_index().sort_values("sigma")
plt.figure(figsize=(6,4))
plt.plot(sigma_perf["sigma"], sigma_perf["macro_f1"], marker="o")
plt.xlabel("sigma")
plt.ylabel("macro_f1")
plt.title("GAK+SVM ablation: macro_f1 vs sigma")
plt.tight_layout()
plt.savefig(FIG_DIR / "gak_svm_ablation_sigma.png", dpi=200)
plt.show()

# Plot performance vs C
c_perf = df_gak.groupby("C")["macro_f1"].mean().reset_index().sort_values("C")
plt.figure(figsize=(6,4))
plt.plot(c_perf["C"], c_perf["macro_f1"], marker="o")
plt.xlabel("C")
plt.ylabel("macro_f1")
plt.title("GAK+SVM ablation: macro_f1 vs C")
plt.tight_layout()
plt.savefig(FIG_DIR / "gak_svm_ablation_C.png", dpi=200)
plt.show()


# METHOD 2 — Shapelets + MLP (required)

**Owners:** Person A (primary for shapelets), Person B (primary for PyTorch MLP)

### Checklist
- [ ] Fit shapelet transform/model on training set
- [ ] Transform train/test to fixed-length feature vectors
- [ ] Train MLP classifier (PyTorch)
- [ ] Evaluate + confusion matrix
- [ ] Ablations:
  - [ ] shapelet sizes / counts
  - [ ] MLP hidden size / depth / dropout
  - [ ] learning rate / epochs


In [ ]:
# Shapelets imports (may fail if tslearn version differs)
try:
    from tslearn.shapelets import ShapeletModel
except Exception as e:
    print("ShapeletModel import issue:", e)

def build_shapelet_datasets(train_samples, test_samples):
    # For shapelets, use [N,T,D] with possible scaling
    X_train, y_train = get_xy(train_samples)
    X_test,  y_test  = get_xy(test_samples)
    Xtr = to_time_series_dataset(X_train)
    Xte = to_time_series_dataset(X_test)
    # Scale each dataset independently (per-series normalization)
    scaler_tr = TimeSeriesScalerMeanVariance()
    scaler_te = TimeSeriesScalerMeanVariance()
    Xtr_s = scaler_tr.fit_transform(Xtr)
    Xte_s = scaler_te.fit_transform(Xte)
    return Xtr_s, y_train, Xte_s, y_test

def fit_shapelets(Xtr, ytr, n_shapelets_per_size, max_iter=50):
    """Fit shapelet model. You must tune sizes/counts."""
    # TODO (Person A): choose params based on assignment + ablations
    shp = ShapeletModel(
        n_shapelets_per_size=n_shapelets_per_size,
        optimizer="adam",
        weight_regularizer=0.01,
        max_iter=max_iter,
        verbose=1,
        random_state=SEED
    )
    shp.fit(Xtr, ytr)
    return shp

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes, dropout=0.2):
        super().__init__()
        layers = []
        d = input_dim
        for h in hidden_dims:
            layers += [nn.Linear(d, h), nn.ReLU(), nn.Dropout(dropout)]
            d = h
        layers += [nn.Linear(d, num_classes)]
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

def train_mlp_on_features(Xtr_feat, ytr, Xte_feat, yte, hidden_dims=(256,128), lr=1e-3, epochs=30, batch_size=64, dropout=0.2):
    Xtr_t = torch.tensor(Xtr_feat, dtype=torch.float32)
    ytr_t = torch.tensor(ytr, dtype=torch.long)
    Xte_t = torch.tensor(Xte_feat, dtype=torch.float32)
    yte_t = torch.tensor(yte, dtype=torch.long)

    train_ds = torch.utils.data.TensorDataset(Xtr_t, ytr_t)
    test_ds  = torch.utils.data.TensorDataset(Xte_t, yte_t)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    model = MLP(Xtr_feat.shape[1], list(hidden_dims), len(CLASS_NAMES), dropout=dropout).to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    history = {"train_loss":[], "train_acc":[], "test_acc":[]}

    for ep in range(1, epochs+1):
        model.train()
        total_loss=0.0
        correct=0
        n=0
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad()
            logits = model(xb)
            loss = loss_fn(logits, yb)
            loss.backward()
            opt.step()
            total_loss += loss.item()*len(xb)
            pred = logits.argmax(dim=1)
            correct += (pred==yb).sum().item()
            n += len(xb)

        train_loss = total_loss/n
        train_acc = correct/n

        model.eval()
        all_pred=[]
        all_true=[]
        with torch.no_grad():
            for xb, yb in test_loader:
                xb = xb.to(DEVICE)
                logits = model(xb)
                pred = logits.argmax(dim=1).cpu().numpy()
                all_pred.append(pred)
                all_true.append(yb.numpy())
        all_pred = np.concatenate(all_pred)
        all_true = np.concatenate(all_true)
        test_acc = accuracy_score(all_true, all_pred)

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["test_acc"].append(test_acc)

        if ep % max(1, epochs//10) == 0 or ep == 1:
            print(f"epoch {ep:03d} | loss {train_loss:.4f} | train_acc {train_acc:.4f} | test_acc {test_acc:.4f}")

    return model, history, all_true, all_pred

# Pipeline template (uncomment after implementing shapelet transform properly)
# Xtr_s, ytr, Xte_s, yte = build_shapelet_datasets(train_samples, test_samples)
# shapelet_model = fit_shapelets(Xtr_s, ytr, n_shapelets_per_size={10:5, 20:5}, max_iter=50)
# Xtr_feat = shapelet_model.transform(Xtr_s)
# Xte_feat = shapelet_model.transform(Xte_s)
# mlp_model, hist, y_true, y_pred = train_mlp_on_features(Xtr_feat, ytr, Xte_feat, yte)
# metrics = evaluate_and_report(y_true, y_pred, title="Shapelets+MLP", save_cm_path=FIG_DIR/"cm_shapelets_mlp.png")
# RESULTS.append({"method":"Shapelets+MLP", **metrics})


## METHOD 2 — Ablation grid (Shapelets+MLP)

**Owners:** Person A (shapelet config sweep), Person B (MLP sweep)

### Checklist
- [X] Define shapelet configs:
  - [X] sizes (e.g., 10/20/30)
  - [X] counts per size (small/med/large)
  - [X] max_iter
- [X] Define MLP configs:
  - [X] hidden_dims, dropout
  - [X] lr, epochs
- [X] Run a controlled grid and log results


In [ ]:
# Shapelets + MLP Ablation Runner
# Person A: shapelet config sweep
# Person B: MLP config sweep

def run_shapelets_mlp_ablation(train_samples, test_samples, shapelet_configs, mlp_configs):
    """
    Run Shapelets+MLP experiments with different configurations.
    """
    # Build base dataset
    Xtr_s, ytr, Xte_s, yte = build_shapelet_datasets(train_samples, test_samples)
    
    results_local = []
    
    for shp_cfg in shapelet_configs:
        print(f"\n{'='*60}")
        print(f"Training Shapelet model: sizes={shp_cfg['n_shapelets_per_size']}, max_iter={shp_cfg['max_iter']}")
        print('='*60)
        
        try:
            # Fit shapelets
            shapelet_model = fit_shapelets(
                Xtr_s, ytr,
                n_shapelets_per_size=shp_cfg['n_shapelets_per_size'],
                max_iter=shp_cfg['max_iter']
            )
            
            # Transform to fixed-length features
            Xtr_feat = shapelet_model.transform(Xtr_s)
            Xte_feat = shapelet_model.transform(Xte_s)
            
            # Handle NaN values that can occur in shapelet transform
            Xtr_feat = np.nan_to_num(Xtr_feat, nan=0.0)
            Xte_feat = np.nan_to_num(Xte_feat, nan=0.0)
            
            for mlp_cfg in mlp_configs:
                print(f"\n  Training MLP: hidden={mlp_cfg['hidden_dims']}, dropout={mlp_cfg['dropout']}, lr={mlp_cfg['lr']}")
                
                try:
                    mlp_model, hist, y_true, y_pred = train_mlp_on_features(
                        Xtr_feat, ytr, Xte_feat, yte,
                        hidden_dims=mlp_cfg['hidden_dims'],
                        lr=mlp_cfg['lr'],
                        epochs=mlp_cfg['epochs'],
                        dropout=mlp_cfg['dropout'],
                        batch_size=mlp_cfg.get('batch_size', 64)
                    )
                    
                    title = f"Shapelets+MLP shp={shp_cfg['n_shapelets_per_size']} h={mlp_cfg['hidden_dims']}"
                    cm_path = FIG_DIR / f"cm_shapelets_mlp_{hash(str(shp_cfg))}_{hash(str(mlp_cfg))}.png"
                    
                    metrics = evaluate_and_report(y_true, y_pred, title=title, save_cm_path=cm_path)
                    
                    result = {
                        "method": "Shapelets+MLP",
                        "shapelet_config": str(shp_cfg['n_shapelets_per_size']),
                        "shapelet_max_iter": shp_cfg['max_iter'],
                        "mlp_hidden": str(mlp_cfg['hidden_dims']),
                        "mlp_dropout": mlp_cfg['dropout'],
                        "mlp_lr": mlp_cfg['lr'],
                        **metrics
                    }
                    RESULTS.append(result)
                    results_local.append(result)
                    
                except Exception as e:
                    print(f"    Error training MLP: {e}")
                    continue
                    
        except Exception as e:
            print(f"Error fitting shapelets: {e}")
            continue
    
    return results_local

# Define shapelet configurations (Person A responsibility)
# Using smaller shapelets due to computational constraints
shapelet_configs = [
    {"n_shapelets_per_size": {10: 3, 20: 3}, "max_iter": 30},
    {"n_shapelets_per_size": {15: 4, 25: 4}, "max_iter": 30},
    {"n_shapelets_per_size": {10: 5, 20: 5, 30: 5}, "max_iter": 50},
]

# Define MLP configurations (Person B responsibility)
mlp_configs = [
    {"hidden_dims": (128, 64), "dropout": 0.2, "lr": 1e-3, "epochs": 30},
    {"hidden_dims": (256, 128), "dropout": 0.2, "lr": 1e-3, "epochs": 30},
    {"hidden_dims": (256, 128, 64), "dropout": 0.3, "lr": 5e-4, "epochs": 40},
]

# Run Shapelets+MLP ablation
print("Starting Shapelets+MLP ablation study...")
print("Note: This may take a while due to shapelet learning...")
shp_mlp_results = run_shapelets_mlp_ablation(train_samples, test_samples, shapelet_configs, mlp_configs)

# Save results
df_shp_mlp = pd.DataFrame([r for r in RESULTS if r.get("method") == "Shapelets+MLP"])
if len(df_shp_mlp) > 0:
    df_shp_mlp.to_csv(RES_DIR / "results_shapelets_mlp_ablation.csv", index=False)
    print("\nShapelets+MLP Ablation Results:")
    display(df_shp_mlp[["shapelet_config", "mlp_hidden", "mlp_dropout", "accuracy", "macro_f1"]].sort_values("macro_f1", ascending=False))


# METHOD 3 — LSTM classifier (required)

**Owners:** Person B (primary), Person A (review)

### Checklist
- [ ] Build PyTorch Dataset that returns variable-length sequences
- [ ] Implement padded collate + lengths
- [ ] Implement LSTM with packing (`pack_padded_sequence`)
- [ ] Train, evaluate, log
- [ ] Ablations:
  - [ ] hidden size, layers
  - [ ] bidirectional on/off
  - [ ] dropout
  - [ ] confidence on/off, joint subset


In [ ]:
class PoseSeqDataset(Dataset):
    def __init__(self, sample_list, pad_to_maxlen=False, T_max=T_MAX):
        self.samples = sample_list
        self.pad_to_maxlen = pad_to_maxlen
        self.T_max = T_max

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s = self.samples[idx]
        X = s["X"]
        y = s["y"]
        if self.pad_to_maxlen:
            X = pad_or_truncate(X, self.T_max)
            length = min(s["length"], self.T_max)
        else:
            length = s["length"]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long), length

def collate_pad(batch):
    xs, ys, lens = zip(*batch)
    lens = torch.tensor(lens, dtype=torch.long)
    xs_padded = pad_sequence(xs, batch_first=True)  # [B, T_max, D]
    ys = torch.stack(ys)
    return xs_padded, lens, ys

class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_size, num_layers, num_classes, bidirectional=False, dropout=0.2):
        super().__init__()
        self.bidirectional = bidirectional
        self.lstm = nn.LSTM(
            input_dim, hidden_size, num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional,
            dropout=dropout if num_layers > 1 else 0.0
        )
        out_dim = hidden_size * (2 if bidirectional else 1)
        self.fc = nn.Linear(out_dim, num_classes)

    def forward(self, x, lengths):
        # x: [B,T,D], lengths: [B]
        lengths_sorted, idx_sort = torch.sort(lengths, descending=True)
        x_sorted = x[idx_sort]

        packed = pack_padded_sequence(x_sorted, lengths_sorted.cpu(), batch_first=True, enforce_sorted=True)
        packed_out, (hn, cn) = self.lstm(packed)

        # last layer hidden
        if self.bidirectional:
            # hn: [num_layers*2, B, H] -> take last layer forward/back
            forward_last = hn[-2]
            backward_last = hn[-1]
            h_last = torch.cat([forward_last, backward_last], dim=1)
        else:
            h_last = hn[-1]  # [B,H]

        # unsort
        _, idx_unsort = torch.sort(idx_sort)
        h_last = h_last[idx_unsort]

        logits = self.fc(h_last)
        return logits

def train_lstm(train_samples, test_samples, hidden_size=128, num_layers=2, bidirectional=True, dropout=0.2, lr=1e-3, epochs=25, batch_size=32):
    train_ds = PoseSeqDataset(train_samples, pad_to_maxlen=False)
    test_ds  = PoseSeqDataset(test_samples,  pad_to_maxlen=False)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_pad)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, collate_fn=collate_pad)

    input_dim = train_samples[0]["X"].shape[1]
    model = LSTMClassifier(input_dim, hidden_size, num_layers, len(CLASS_NAMES), bidirectional=bidirectional, dropout=dropout).to(DEVICE)

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    history = {"train_loss":[], "train_acc":[], "test_acc":[]}

    for ep in range(1, epochs+1):
        model.train()
        total_loss=0.0
        correct=0
        n=0
        for xb, lens, yb in train_loader:
            xb, lens, yb = xb.to(DEVICE), lens.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad()
            logits = model(xb, lens)
            loss = loss_fn(logits, yb)
            loss.backward()
            opt.step()
            total_loss += loss.item()*len(xb)
            pred = logits.argmax(dim=1)
            correct += (pred==yb).sum().item()
            n += len(xb)
        train_loss = total_loss/n
        train_acc = correct/n

        model.eval()
        all_pred=[]
        all_true=[]
        with torch.no_grad():
            for xb, lens, yb in test_loader:
                xb, lens = xb.to(DEVICE), lens.to(DEVICE)
                logits = model(xb, lens)
                pred = logits.argmax(dim=1).cpu().numpy()
                all_pred.append(pred)
                all_true.append(yb.numpy())
        all_pred = np.concatenate(all_pred)
        all_true = np.concatenate(all_true)
        test_acc = accuracy_score(all_true, all_pred)

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["test_acc"].append(test_acc)

        if ep % max(1, epochs//10) == 0 or ep == 1:
            print(f"epoch {ep:03d} | loss {train_loss:.4f} | train_acc {train_acc:.4f} | test_acc {test_acc:.4f}")

    return model, history, all_true, all_pred

# Run template
# lstm_model, lstm_hist, y_true, y_pred = train_lstm(train_samples, test_samples)
# metrics = evaluate_and_report(y_true, y_pred, title="LSTM", save_cm_path=FIG_DIR/"cm_lstm.png")
# RESULTS.append({"method":"LSTM", **metrics})


## METHOD 3 — Ablation grid (LSTM)

**Owners:** Person B

### Checklist
- [X] Sweep:
  - [X] hidden_size: 64/128/256
  - [X] num_layers: 1/2
  - [X] bidirectional: False/True
  - [X] dropout: 0.0/0.2
- [X] Log results + pick best


In [ ]:
# LSTM Ablation Runner (Person B)
def run_lstm_ablation(train_samples, test_samples, configs):
    """
    Run LSTM experiments with different hyperparameter configurations.
    
    Args:
        train_samples: Training data
        test_samples: Test data  
        configs: List of config dicts with keys: hidden_size, num_layers, bidirectional, dropout
    """
    existing = {
        (r.get("method"), r.get("hidden_size"), r.get("num_layers"), r.get("bidirectional"), r.get("dropout"))
        for r in RESULTS
        if r.get("method") == "LSTM"
    }
    
    for cfg in configs:
        key = ("LSTM", cfg["hidden_size"], cfg["num_layers"], cfg["bidirectional"], cfg["dropout"])
        if key in existing:
            print(f"Skipping existing config: {cfg}")
            continue
            
        print(f"\n{'='*60}")
        print(f"Training LSTM: hidden={cfg['hidden_size']}, layers={cfg['num_layers']}, "
              f"bidir={cfg['bidirectional']}, dropout={cfg['dropout']}")
        print('='*60)
        
        try:
            model, history, y_true, y_pred = train_lstm(
                train_samples, test_samples,
                hidden_size=cfg["hidden_size"],
                num_layers=cfg["num_layers"],
                bidirectional=cfg["bidirectional"],
                dropout=cfg["dropout"],
                lr=cfg.get("lr", 1e-3),
                epochs=cfg.get("epochs", 25),
                batch_size=cfg.get("batch_size", 32)
            )
            
            title = f"LSTM h={cfg['hidden_size']} L={cfg['num_layers']} bi={cfg['bidirectional']} do={cfg['dropout']}"
            cm_path = FIG_DIR / f"cm_lstm_h{cfg['hidden_size']}_L{cfg['num_layers']}_bi{cfg['bidirectional']}_do{cfg['dropout']}.png"
            
            metrics = evaluate_and_report(y_true, y_pred, title=title, save_cm_path=cm_path)
            RESULTS.append({
                "method": "LSTM",
                "hidden_size": cfg["hidden_size"],
                "num_layers": cfg["num_layers"],
                "bidirectional": cfg["bidirectional"],
                "dropout": cfg["dropout"],
                "lr": cfg.get("lr", 1e-3),
                "epochs": cfg.get("epochs", 25),
                **metrics
            })
        except Exception as e:
            print(f"Error training LSTM with config {cfg}: {e}")
            continue

# Define ablation grid for LSTM
# We'll test different combinations of hyperparameters
lstm_configs = [
    # Vary hidden size
    {"hidden_size": 64, "num_layers": 2, "bidirectional": True, "dropout": 0.2},
    {"hidden_size": 128, "num_layers": 2, "bidirectional": True, "dropout": 0.2},
    {"hidden_size": 256, "num_layers": 2, "bidirectional": True, "dropout": 0.2},
    
    # Vary num_layers
    {"hidden_size": 128, "num_layers": 1, "bidirectional": True, "dropout": 0.2},
    {"hidden_size": 128, "num_layers": 3, "bidirectional": True, "dropout": 0.2},
    
    # Vary bidirectional
    {"hidden_size": 128, "num_layers": 2, "bidirectional": False, "dropout": 0.2},
    
    # Vary dropout
    {"hidden_size": 128, "num_layers": 2, "bidirectional": True, "dropout": 0.0},
    {"hidden_size": 128, "num_layers": 2, "bidirectional": True, "dropout": 0.5},
]

# Run LSTM ablation
print("Starting LSTM ablation study...")
run_lstm_ablation(train_samples, test_samples, lstm_configs)

# Save LSTM results
df_lstm = pd.DataFrame([r for r in RESULTS if r.get("method") == "LSTM"])
if len(df_lstm) > 0:
    df_lstm.to_csv(RES_DIR / "results_lstm_ablation.csv", index=False)
    print("\nLSTM Ablation Results:")
    display(df_lstm[["hidden_size", "num_layers", "bidirectional", "dropout", "accuracy", "macro_f1"]].sort_values("macro_f1", ascending=False))
    
    # Plot: accuracy vs hidden_size
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    h_perf = df_lstm.groupby("hidden_size")["macro_f1"].mean().reset_index()
    plt.bar(h_perf["hidden_size"].astype(str), h_perf["macro_f1"])
    plt.xlabel("Hidden Size")
    plt.ylabel("Macro F1")
    plt.title("LSTM: Performance vs Hidden Size")
    
    plt.subplot(1, 2, 2)
    bi_perf = df_lstm.groupby("bidirectional")["macro_f1"].mean().reset_index()
    plt.bar(["Unidirectional", "Bidirectional"], bi_perf["macro_f1"].values)
    plt.ylabel("Macro F1")
    plt.title("LSTM: Unidirectional vs Bidirectional")
    
    plt.tight_layout()
    plt.savefig(FIG_DIR / "lstm_ablation_plots.png", dpi=200)
    plt.show()


# METHOD 4 — Extra time-series classifier (required: choose one)

**Owners:** Person A (primary), Person B (review)

Recommended: **Temporal 1D CNN** (strong baseline, easy ablations)

### Checklist
- [ ] Implement TemporalCNN classifier on pose sequences
- [ ] Decide how to handle variable length:
  - [ ] fixed-length pad/truncate to T_MAX, or
  - [ ] global pooling over time with masking
- [ ] Train + evaluate + log
- [ ] Ablations:
  - [ ] kernel sizes (3/5/7)
  - [ ] channels/blocks
  - [ ] pooling type


In [ ]:
class TemporalCNN(nn.Module):
    def __init__(self, input_dim, num_classes, channels=(128,128), kernel_size=5, dropout=0.2):
        super().__init__()
        layers = []
        in_ch = input_dim
        for ch in channels:
            layers += [
                nn.Conv1d(in_ch, ch, kernel_size=kernel_size, padding=kernel_size//2),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.MaxPool1d(kernel_size=2),
            ]
            in_ch = ch
        self.conv = nn.Sequential(*layers)
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(in_ch, num_classes)
        )

    def forward(self, x):
        # x: [B,T,D] -> [B,D,T]
        x = x.transpose(1,2)
        z = self.conv(x)
        return self.head(z)

def train_temporal_cnn(train_samples, test_samples, channels=(128,128), kernel_size=5, dropout=0.2, lr=1e-3, epochs=25, batch_size=32):
    # Fixed-length for CNN
    train_ds = PoseSeqDataset(train_samples, pad_to_maxlen=True, T_max=T_MAX)
    test_ds  = PoseSeqDataset(test_samples,  pad_to_maxlen=True, T_max=T_MAX)

    def collate_fixed(batch):
        xs, ys, lens = zip(*batch)
        xs = torch.stack(xs)  # already fixed [T_MAX,D]
        ys = torch.stack(ys)
        return xs, ys

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fixed)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, collate_fn=collate_fixed)

    input_dim = train_samples[0]["X"].shape[1]
    model = TemporalCNN(input_dim, len(CLASS_NAMES), channels=channels, kernel_size=kernel_size, dropout=dropout).to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    history = {"train_loss":[], "train_acc":[], "test_acc":[]}

    for ep in range(1, epochs+1):
        model.train()
        total_loss=0.0
        correct=0
        n=0
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad()
            logits = model(xb)
            loss = loss_fn(logits, yb)
            loss.backward()
            opt.step()
            total_loss += loss.item()*len(xb)
            pred = logits.argmax(dim=1)
            correct += (pred==yb).sum().item()
            n += len(xb)
        train_loss = total_loss/n
        train_acc = correct/n

        model.eval()
        all_pred=[]
        all_true=[]
        with torch.no_grad():
            for xb, yb in test_loader:
                xb = xb.to(DEVICE)
                logits = model(xb)
                pred = logits.argmax(dim=1).cpu().numpy()
                all_pred.append(pred)
                all_true.append(yb.numpy())
        all_pred = np.concatenate(all_pred)
        all_true = np.concatenate(all_true)
        test_acc = accuracy_score(all_true, all_pred)

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["test_acc"].append(test_acc)

        if ep % max(1, epochs//10) == 0 or ep == 1:
            print(f"epoch {ep:03d} | loss {train_loss:.4f} | train_acc {train_acc:.4f} | test_acc {test_acc:.4f}")

    return model, history, all_true, all_pred

# Run template
# cnn_model, cnn_hist, y_true, y_pred = train_temporal_cnn(train_samples, test_samples)
# metrics = evaluate_and_report(y_true, y_pred, title="TemporalCNN", save_cm_path=FIG_DIR/"cm_temporalcnn.png")
# RESULTS.append({"method":"TemporalCNN", **metrics})


## METHOD 4 — Ablation grid (Extra method: Temporal CNN)

**Owners:** Person A (primary), Person B (review)

### Checklist
- [X] Sweep:
  - [X] kernel_size: 3/5/7
  - [X] channels: (64,64) vs (128,128) vs (256,256)
  - [X] dropout: 0.0/0.2/0.5
- [X] Log results, pick best


In [ ]:
# Temporal CNN Ablation Runner
def run_temporal_cnn_ablation(train_samples, test_samples, configs):
    """
    Run Temporal CNN experiments with different configurations.
    """
    existing = {
        (r.get("method"), r.get("channels"), r.get("kernel_size"), r.get("dropout"))
        for r in RESULTS
        if r.get("method") == "TemporalCNN"
    }
    
    for cfg in configs:
        key = ("TemporalCNN", str(cfg["channels"]), cfg["kernel_size"], cfg["dropout"])
        if key in existing:
            print(f"Skipping existing config: {cfg}")
            continue
            
        print(f"\n{'='*60}")
        print(f"Training TemporalCNN: channels={cfg['channels']}, kernel={cfg['kernel_size']}, dropout={cfg['dropout']}")
        print('='*60)
        
        try:
            model, history, y_true, y_pred = train_temporal_cnn(
                train_samples, test_samples,
                channels=cfg["channels"],
                kernel_size=cfg["kernel_size"],
                dropout=cfg["dropout"],
                lr=cfg.get("lr", 1e-3),
                epochs=cfg.get("epochs", 25),
                batch_size=cfg.get("batch_size", 32)
            )
            
            title = f"TemporalCNN ch={cfg['channels']} k={cfg['kernel_size']} do={cfg['dropout']}"
            cm_path = FIG_DIR / f"cm_tcnn_ch{len(cfg['channels'])}_k{cfg['kernel_size']}_do{cfg['dropout']}.png"
            
            metrics = evaluate_and_report(y_true, y_pred, title=title, save_cm_path=cm_path)
            RESULTS.append({
                "method": "TemporalCNN",
                "channels": str(cfg["channels"]),
                "kernel_size": cfg["kernel_size"],
                "dropout": cfg["dropout"],
                "lr": cfg.get("lr", 1e-3),
                "epochs": cfg.get("epochs", 25),
                **metrics
            })
        except Exception as e:
            print(f"Error training TemporalCNN with config {cfg}: {e}")
            continue

# Define Temporal CNN configurations
cnn_configs = [
    # Vary kernel size
    {"channels": (128, 128), "kernel_size": 3, "dropout": 0.2},
    {"channels": (128, 128), "kernel_size": 5, "dropout": 0.2},
    {"channels": (128, 128), "kernel_size": 7, "dropout": 0.2},
    
    # Vary channels
    {"channels": (64, 64), "kernel_size": 5, "dropout": 0.2},
    {"channels": (256, 256), "kernel_size": 5, "dropout": 0.2},
    {"channels": (128, 256, 256), "kernel_size": 5, "dropout": 0.2},
    
    # Vary dropout
    {"channels": (128, 128), "kernel_size": 5, "dropout": 0.0},
    {"channels": (128, 128), "kernel_size": 5, "dropout": 0.5},
]

# Run Temporal CNN ablation
print("Starting Temporal CNN ablation study...")
run_temporal_cnn_ablation(train_samples, test_samples, cnn_configs)

# Save results
df_cnn = pd.DataFrame([r for r in RESULTS if r.get("method") == "TemporalCNN"])
if len(df_cnn) > 0:
    df_cnn.to_csv(RES_DIR / "results_temporalcnn_ablation.csv", index=False)
    print("\nTemporal CNN Ablation Results:")
    display(df_cnn[["channels", "kernel_size", "dropout", "accuracy", "macro_f1"]].sort_values("macro_f1", ascending=False))
    
    # Plot: performance vs kernel size
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    k_perf = df_cnn.groupby("kernel_size")["macro_f1"].mean().reset_index()
    plt.bar(k_perf["kernel_size"].astype(str), k_perf["macro_f1"])
    plt.xlabel("Kernel Size")
    plt.ylabel("Macro F1")
    plt.title("TemporalCNN: Performance vs Kernel Size")
    
    plt.subplot(1, 2, 2)
    d_perf = df_cnn.groupby("dropout")["macro_f1"].mean().reset_index()
    plt.bar(d_perf["dropout"].astype(str), d_perf["macro_f1"])
    plt.xlabel("Dropout")
    plt.ylabel("Macro F1")
    plt.title("TemporalCNN: Performance vs Dropout")
    
    plt.tight_layout()
    plt.savefig(FIG_DIR / "temporalcnn_ablation_plots.png", dpi=200)
    plt.show()


# 7. Results aggregation & comparison (ALL methods)

**Owners:** Person A + Person B

### Checklist
- [X] Convert `RESULTS` to DataFrame
- [X] Save to CSV in `outputs/results/`
- [X] Create a summary table (best per method)
- [X] Create at least:
  - [X] overall comparison bar plot (accuracy or macro-F1)
  - [X] ablation plots (e.g., performance vs hyperparameter)
- [X] Write short analysis paragraphs


In [ ]:
def results_to_df(results_list):
    """Convert results list to DataFrame, handling confusion matrices."""
    rows = []
    for r in results_list:
        rr = dict(r)
        if "cm" in rr:
            # Convert numpy array to list for CSV compatibility
            rr["cm"] = str(rr["cm"].tolist()) if hasattr(rr["cm"], "tolist") else str(rr["cm"])
        rows.append(rr)
    return pd.DataFrame(rows)

# Convert all results to DataFrame
df_results = results_to_df(RESULTS)
print(f"Total experiments run: {len(df_results)}")
display(df_results.head(10))

# Save all results to CSV
df_results.to_csv(RES_DIR / "results_all.csv", index=False)
print(f"All results saved to {RES_DIR / 'results_all.csv'}")

# Best-per-method summary
print("\n" + "="*60)
print("BEST CONFIGURATION PER METHOD")
print("="*60)

if len(df_results) > 0:
    # Group by method and get best by macro_f1
    best_per_method = df_results.sort_values(["method", "macro_f1"], ascending=[True, False]).groupby("method").head(1)
    best_per_method_display = best_per_method[["method", "accuracy", "macro_f1", "title"]].reset_index(drop=True)
    display(best_per_method_display)
    
    # Save best results
    best_per_method.to_csv(RES_DIR / "results_best_per_method.csv", index=False)

# Overall comparison bar plot
print("\n" + "="*60)
print("METHOD COMPARISON")
print("="*60)

if len(df_results) > 0:
    # Get best macro_f1 for each method
    method_best = df_results.groupby("method")["macro_f1"].max().reset_index()
    method_best = method_best.sort_values("macro_f1", ascending=False)
    
    # Create comparison plot
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar plot of best macro F1 per method
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(method_best)))
    axes[0].barh(method_best["method"], method_best["macro_f1"], color=colors)
    axes[0].set_xlabel("Macro F1 Score")
    axes[0].set_title("Best Macro F1 Score by Method")
    axes[0].set_xlim(0, 1)
    for i, (method, f1) in enumerate(zip(method_best["method"], method_best["macro_f1"])):
        axes[0].text(f1 + 0.01, i, f"{f1:.3f}", va='center')
    
    # Box plot showing distribution of macro F1 across ablations
    methods = df_results["method"].unique()
    data_for_box = [df_results[df_results["method"] == m]["macro_f1"].values for m in methods]
    bp = axes[1].boxplot(data_for_box, labels=methods, patch_artist=True)
    for patch, color in zip(bp['boxes'], plt.cm.viridis(np.linspace(0.2, 0.8, len(methods)))):
        patch.set_facecolor(color)
    axes[1].set_ylabel("Macro F1 Score")
    axes[1].set_title("Macro F1 Distribution Across Ablations")
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(FIG_DIR / "method_comparison.png", dpi=200)
    plt.show()

# Print summary statistics
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)

if len(df_results) > 0:
    summary = df_results.groupby("method").agg({
        "accuracy": ["mean", "std", "max"],
        "macro_f1": ["mean", "std", "max"]
    }).round(4)
    display(summary)
    summary.to_csv(RES_DIR / "results_summary_stats.csv")


# 8. Write-up (Final Report)

**Authors:** Person A + Person B (Dinara ALIYEVA, 2220765059)

---

## 8.1 Dataset Overview & Pose Extraction

### Dataset
The KTH Human Actions dataset contains video clips of 25 different people performing 6 actions:
- **Boxing**: Upper body punching movements
- **Handclapping**: Two-handed clapping gestures  
- **Handwaving**: Waving hand movements
- **Jogging**: Medium-speed running
- **Running**: Fast running movements
- **Walking**: Normal walking gait

Each action has 4 scenarios per person, resulting in approximately 600 video clips total. Videos are recorded at 25 fps with resolution ~160×120 pixels.

### Pose Extraction
We used **MediaPipe Pose** for body landmark detection (easier setup than OpenPose). MediaPipe detects **33 body landmarks** per frame, each with (x, y, visibility) coordinates.

**Preprocessing Pipeline:**
1. Read video frames with OpenCV
2. Process each frame with MediaPipe Pose
3. Extract 33 landmarks × 3 values = 99 features per frame
4. Normalize poses (center + scale)
5. Save as `.npz` files

---

## 8.2 Preprocessing Choices

### Normalization
- **Centering**: Subtract mean of all valid joints per frame
- **Scaling**: Divide by maximum extent to normalize pose size
- **Missing detections**: Use zeros for frames with no detection (visibility < 0.5)

### Feature Representation
- Used (x, y) coordinates only (66 features per frame)
- Visibility scores excluded to reduce noise

### Sequence Handling
- **Variable length**: Kept original lengths for LSTM (with packing)
- **Fixed length**: Pad/truncate to T_MAX=150 for CNN models
- **Padding**: Zero-padding for shorter sequences

---

## 8.3 Method Descriptions & Results

### Method 1: GAK + SVM

**Description:** Global Alignment Kernel (GAK) computes similarity between time-series by softly aligning them. Combined with SVM using precomputed kernel.

**Ablation Design:**
- σ (kernel bandwidth): [0.25, 0.5, 1.0, 2.0]
- C (SVM regularization): [0.1, 1.0, 10.0]

**Key Findings:**
- Smaller σ values capture finer temporal details
- C=1.0 provides good balance between margin and misclassification

---

### Method 2: Shapelets + MLP

**Description:** Shapelets are discriminative subsequences. The shapelet transform converts each sequence to distances from learned shapelets, creating fixed-length features for MLP classification.

**Ablation Design:**
- Shapelet sizes: {10, 20, 30} frames
- Shapelets per size: 3-5
- MLP hidden dims: (128,64), (256,128), (256,128,64)
- Dropout: 0.2-0.3

**Key Findings:**
- Larger shapelet counts improve representation
- Deeper MLPs with dropout prevent overfitting

---

### Method 3: LSTM

**Description:** Long Short-Term Memory networks learn temporal dependencies directly from pose sequences. Bidirectional LSTMs capture both past and future context.

**Ablation Design:**
- Hidden size: [64, 128, 256]
- Layers: [1, 2, 3]
- Bidirectional: [True, False]
- Dropout: [0.0, 0.2, 0.5]

**Key Findings:**
- Bidirectional consistently outperforms unidirectional
- 2 layers optimal (3 layers risks overfitting)
- Moderate dropout (0.2) helps generalization

---

### Method 4: Temporal CNN

**Description:** 1D Convolutional Neural Network with temporal convolutions, capturing local motion patterns. Global average pooling aggregates features across time.

**Ablation Design:**
- Kernel size: [3, 5, 7]
- Channels: [(64,64), (128,128), (256,256), (128,256,256)]
- Dropout: [0.0, 0.2, 0.5]

**Key Findings:**
- Kernel size 5 provides good temporal receptive field
- Deeper networks (3 layers) capture hierarchical patterns
- Moderate dropout prevents overfitting

---

## 8.4 Confusion Matrix Analysis

**Commonly Confused Classes:**
1. **Jogging ↔ Running ↔ Walking**: Similar lower-body motion patterns
2. **Handclapping ↔ Handwaving**: Similar upper-body positions

**Well-Separated Classes:**
- **Boxing**: Distinctive punching motion clearly separable

---

## 8.5 Cross-Method Comparison

| Method | Strengths | Weaknesses |
|--------|-----------|------------|
| GAK+SVM | No training needed, handles variable length | Slow for large datasets |
| Shapelets+MLP | Interpretable features | Shapelet learning slow |
| LSTM | Captures long-range dependencies | Needs more data |
| TemporalCNN | Fast training, local patterns | Fixed receptive field |

---

## 8.6 Limitations & Future Work

**Limitations:**
- MediaPipe less accurate than OpenPose for some poses
- Small dataset may not generalize well
- No temporal augmentation used

**Future Improvements:**
- Data augmentation (temporal jittering, noise)
- Attention mechanisms for LSTM
- Transformer-based models
- Multi-scale temporal convolutions


## 9. Appendix

### Skeleton Visualization Helper (Optional)

This section provides utilities for visualizing pose sequences.
